In [1]:
# ── cell: 1 ──
# 📁 Prepare Workspace


MODEL_NAME= "moussaKam_AraBART" # "fewshot_gemini" #"agemagician_mlong-t5-tglobal-large"  #"UBC-NLP_AraT5-base"
MODEL_TYPE= "fine_tuned_model"

import os

path = "/kaggle/working"
is_empty = (len(os.listdir(path)) <= 3)

# !cp -r /kaggle/input/arageneval2025-task1-sc/* /kaggle/working/

if is_empty:
    print(f"✅ '{path}' is empty. copying project strcuture into it ...")
    !cp -r /kaggle/input/arageneval2025-task1-sc/* /kaggle/working/
    !mkdir -p /kaggle/working/evaluation/ar_style_classifier/
    !cp -r /kaggle/input/binary-style-classifier-models-21-authors/working/* /kaggle/working/evaluation/ar_style_classifier/
    !cp -r /kaggle/input/ast-fine-tuned-models-generations/results/results/* /kaggle/working/models/fine_tuned_model/results/

    !mkdir -p /kaggle/working/models/baseline_model/results/fewshot_gemini
    !cp -r /kaggle/input/ast-fine-tuned-models-generations/baseline_model/baseline_model/results/fewshot_gemini/generation_log.json /kaggle/working/models/baseline_model/results/fewshot_gemini
else:
    print(f"❌ '{path}' is not empty — contains {len(os.listdir(path))} items.")


# !cp -r /kaggle/input/binary-style-classifier-models-21-authors/working/* /kaggle/working/evaluation/ar_style_classifier/
!cp -r /kaggle/input/ast-fine-tuned-models-generations/results/results/* /kaggle/working/models/fine_tuned_model/results/



# Change to project directory
%cd /kaggle/working/ 

# Confirm
!ls

❌ '/kaggle/working' is not empty — contains 11 items.
/kaggle/working
config.py  evaluation	models	 __pycache__	  state.db
data	   __init__.py	prompts  run_pipeline.py  utils


In [2]:
!ls -R /kaggle/working/models/MODEL_TYPE}/results/
!ls -R /kaggle/working/models/{MODEL_TYPE}/results/{MODEL_NAME}

ls: cannot access '/kaggle/working/models/MODEL_TYPE}/results/': No such file or directory
/kaggle/working/models/fine_tuned_model/results/google_mt5-small:
generation_log_ERROR.json  generation_log.json


# Dependencies

In [3]:
!pip install --upgrade transformers==4.41.2 datasets==2.19.1 scikit-learn wandb python-dotenv evaluate --quiet
!pip install -U peft==0.11.1 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 89.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 110.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 88.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 85.3 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.7.0 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, 

In [4]:
import sys
import os
import torch

project_root = os.getcwd()  # Should be /content/drive/MyDrive/AraGenEval2025
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Project root added to sys.path: {project_root}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

from config import Config

print(Config.VAL_FILE)

Project root added to sys.path: /kaggle/working
Using device: cuda
data/AuthorshipStyleTransferVal.xlsx


# Mapping

In [5]:
# ── cell: 3 ──
# 🗺 Author Mappings
author2id = {
    "أحمد أمين": 0,  "أحمد تيمور باشا": 1,  "أحمد شوقي": 2,
    "أمين الريحاني": 3,  "ثروت أباظة": 4,  "جبران خليل جبران": 5,
    "جُرجي زيدان": 6,  "حسن حنفي": 7,  "روبرت بار": 8,
    "سلامة موسى": 9,  "طه حسين": 10, "عباس محمود العقاد": 11,
    "عبد الغفار مكاوي": 12, "غوستاف لوبون": 13,  "فؤاد زكريا": 14,
    "كامل كيلاني": 15,  "محمد حسين هيكل": 16,  "نجيب محفوظ": 17,
    "نوال السعداوي": 18, "ويليام شيكسبير": 19, "يوسف إدريس": 20
}
id2author   = {v: k for k, v in author2id.items()}
id2english  = {
    0:"Ahmed_Amin",    1:"Ahmad_Taymour_Basha",   2:"Ahmed_Shawqi",
    3:"Ameen_Rihani",  4:"Tharwat_Abaza",         5:"Gibran_Khalil_Gibran",
    6:"Jurji_Zaydan",  7:"Hassan_Hanifi",         8:"Robert_Barr",
    9:"Salama_Moussa",10:"Taha_Hussein",         11:"Abbas_Al-Aqqad",
   12:"AbdelGhaffar_Makawi",13:"Gustave_Lebon",    14:"Fouad_Zakaria",
   15:"Kamel_Kilani",16:"Mohamed_Hosseini_Hekal",17:"Naguib_Mahfouz",
   18:"Nawal_El_Saadawi",19:"William_Shakespeare",20:"Youssef_Edrees"
}


# Load Dataset

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
import gc

def load_and_sample_dataset(
    file_path,
    sample_mode="all",    # Options: "all", "random", "stratified"
    sample_size=None,     # For "random"/"stratified": float (proportion) or int (absolute count)
    random_state=42
):
    """
    Load dataset from Excel and apply sampling.

    Args:
        file_path (str): Path to dataset file.
        sample_mode (str): "all", "random", or "stratified".
        sample_size (float|int): Proportion (0-1) or count for sampling.
        random_state (int): Random seed for reproducibility.

    Returns:
        pd.DataFrame: Sampled dataset.
    """
    print(f"📂 Loading dataset from: {file_path}")
    df = pd.read_excel(file_path, engine='openpyxl')

    if sample_mode == "all":
        sampled_df = df.copy()
        print(f"✅ Loaded full dataset with {len(sampled_df)} samples.")

    elif sample_mode == "random":
        if sample_size is None:
            raise ValueError("For 'random' mode, SAMPLE_SIZE must be set.")
        sampled_df = df.sample(
            n=sample_size if isinstance(sample_size, int) else int(len(df) * sample_size),
            random_state=random_state
        )
        print(f"✅ Randomly sampled {len(sampled_df)} samples.")

    elif sample_mode == "stratified":
        if sample_size is None or not (0 < sample_size < 1):
            raise ValueError("For 'stratified' mode, SAMPLE_SIZE must be a proportion between 0 and 1.")
        sampled_df, _ = train_test_split(
            df,
            train_size=sample_size,
            stratify=df['author'],
            random_state=random_state
        )
        print(f"✅ Stratified sampled {len(sampled_df)} samples (author distribution preserved).")

    else:
        raise ValueError(f"Invalid sample_mode: {sample_mode}")

    del df
    gc.collect()

    return sampled_df

In [7]:
# === CONFIG ===
SAMPLE_MODE = "stratified"  # "all", "random", "stratified"
SAMPLE_SIZE = 0.1           # 10% for stratified or random. Set to None for full dataset.

# === LOAD AND SAMPLE ===
val_df = load_and_sample_dataset(
    Config.VAL_FILE,
    # sample_mode=SAMPLE_MODE,
    # sample_size=SAMPLE_SIZE
)

# Expect columns: id, text_in_msa
val_df = val_df.rename(columns={"id":"sample_id","text_in_msa":"neutral_text"})
print(f"✅ Loaded {len(val_df)} neutral texts.")

print("Validation shape:", val_df.shape)
print("Authors in Val:", val_df['author'].nunique())

# train_df.head()

📂 Loading dataset from: data/AuthorshipStyleTransferVal.xlsx
✅ Loaded full dataset with 4157 samples.
✅ Loaded 4157 neutral texts.
Validation shape: (4157, 4)
Authors in Val: 21


## Sliding Window

In [8]:
# ── cell: 6 ──
# 🪓 Sliding Window Chunking Utility
from transformers import AutoTokenizer
import torch

def get_chunks(text, tokenizer, max_len=512, stride=256):
    return tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=max_len,
        stride=stride,
        return_overflowing_tokens=True,
        return_tensors="pt"
    )

def device():
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Load Model 

In [9]:
# ── cell: 7 ──
# 🔧 Load Classifier Function
from transformers import AutoModelForSequenceClassification

def load_binary_classifier(author_id):
    path = f"evaluation/ar_style_classifier/author_SC_{author_id}"
    tokenizer = AutoTokenizer.from_pretrained(path)
    model = AutoModelForSequenceClassification.from_pretrained(path)
    model.to(device())
    model.eval()
    return tokenizer, model

## Preparations

In [10]:
# ── cell: 5 ──
# 📝 Load Generation Log
import json
with open(f"models/{MODEL_TYPE}/results/{MODEL_NAME}/generation_log.json","r", encoding="utf-8") as f:
    gen = json.load(f)
# gen: list of {id, author, output, ground_truth}
log_df = pd.DataFrame(gen)
log_df = log_df.rename(columns={"id":"sample_id","output":"generated_text","author":"true_author"})
print(f"✅ Loaded {len(log_df)} generated entries.")

# Merge neutral and generated
df = pd.merge(log_df, val_df, on="sample_id", how="left")
df["true_id"] = df["true_author"].map(author2id)
print(df.head())

✅ Loaded 4157 generated entries.
   sample_id true_author                                     generated_text  \
0       3835  يوسف إدريس  ومنذ أن كنت طفلًا في الخمسين من عمري لم أقم مط...   
1       3836  يوسف إدريس  وذلك الزمن العام هو العداد الجماعي الذي يسجل ا...   
2       3837  يوسف إدريس  ومصر الغنية الراقية والمصنِّعة والعالم العربي ...   
3       3838  يوسف إدريس  ونظرًا لغرابة تجربتي، وللأفكار التي راودتني حو...   
4       3839  يوسف إدريس  ولكن ما ذكره ليس مرارةً ولا ندمًا؛ فقد كان ما ...   

                                        ground_truth  \
0  من طفل في الخمسين\n\nعمري ما احتفلت أو حفلت بع...   
1  ذلك الزمن العام هو العداد العام الذي\n\nدام يع...   
2  مصر الغنية المثقفة المصنِّعة، والعرب\n\nوقد\n\...   
3  ولأنها غريبة وراودتني فيها عن الناس وعن الحياة...   
4  وليس ما ذكرته مرارة ولا ندمًا؛ فقد كان لا يمكن...   

                                        neutral_text  \
0  منذ أن كنت طفلاً في الخمسين من عمري، لم أقم مط...   
1  ذلك الزمن العام يُعتبر بمثابة العداد الج

# Prediction

In [11]:
# ── cell: 8 ──
# 🔍 Probability Prediction (in-style)
import numpy as np

# def predict_instyle_prob(text, tokenizer, model):
#     enc = get_chunks(text, tokenizer)
#     enc = {k: v.to(device()) for k,v in enc.items()}
#     with torch.no_grad():
#         logits = model(**enc).logits  # [chunks, 2]
#         probs = torch.softmax(logits, dim=-1)  # same shape
#     instyle_probs = probs[:,1]  # author style class
#     return instyle_probs.mean().item()


VALID_KEYS = {"input_ids", "attention_mask", "token_type_ids"}

def predict_instyle_prob(text, tokenizer, model):
    enc = get_chunks(text, tokenizer)
    enc = {k: v.to(device()) for k,v in enc.items() if k in VALID_KEYS}
    with torch.no_grad():
        logits = model(**enc).logits  # Now works
        probs = torch.softmax(logits, dim=-1)
    return probs[:,1].mean().item() # author style class


In [12]:
# # ── cell: 9 ──
# # 📊 Style Evaluation Loop
# from tqdm import tqdm
# results = []
# for author_id in range(21):
#     tokenizer, model = load_binary_classifier(author_id)
#     for row in tqdm(df.itertuples(), desc=f"Author {author_id}"):
#         px = predict_instyle_prob(row.neutral_text, tokenizer, model)
#         py = predict_instyle_prob(row.generated_text, tokenizer, model)
#         diff = py - px
#         success = diff > 0
#         results.append({
#             "sample_id": row.sample_id,
#             "true_id": row.true_id,
#             "classifier_id": author_id,
#             "neutral_prob": px,
#             "generated_prob": py,
#             "diff": diff,
#             "success": int(success)
#         })
# results_df = pd.DataFrame(results)



# ── cell: 9 ──
# 📊 Style Evaluation Loop (per‑author filtering)
from tqdm import tqdm

results = []

for author_id in range(21):
    # 1) select only the samples meant for this author
    author_df = df[df["true_id"] == author_id]

    # 2) load that author’s classifier
    tokenizer, model = load_binary_classifier(author_id)

    # 3) run only their own samples
    for row in tqdm(author_df.itertuples(), desc=f"Author {author_id}"):
        px   = predict_instyle_prob(row.neutral_text,   tokenizer, model)
        py   = predict_instyle_prob(row.generated_text, tokenizer, model)
        diff = py - px
        success = int(diff > 0)

        results.append({
            "sample_id":      row.sample_id,
            "true_id":        row.true_id,
            "classifier_id":  author_id,
            "neutral_prob":   px,
            "generated_prob": py,
            "diff":           diff,
            "success":        success
        })

# build DataFrame after the loop
results_df = pd.DataFrame(results)

Author 0: 246it [00:12, 20.03it/s]
Author 1: 57it [00:03, 16.93it/s]
Author 2: 58it [00:03, 19.18it/s]
Author 3: 142it [00:07, 19.11it/s]
Author 4: 90it [00:05, 17.54it/s]
Author 5: 30it [00:01, 17.91it/s]
Author 6: 327it [00:14, 22.23it/s]
Author 7: 548it [00:26, 20.77it/s]
Author 8: 82it [00:03, 24.66it/s]
Author 9: 119it [00:05, 21.30it/s]
Author 10: 255it [00:11, 22.23it/s]
Author 11: 267it [00:11, 22.88it/s]
Author 12: 396it [00:18, 21.13it/s]
Author 13: 150it [00:06, 23.54it/s]
Author 14: 125it [00:05, 24.46it/s]
Author 15: 25it [00:01, 21.31it/s]
Author 16: 260it [00:11, 22.65it/s]
Author 17: 327it [00:16, 19.63it/s]
Author 18: 295it [00:12, 22.81it/s]
Author 19: 238it [00:11, 21.39it/s]
Author 20: 120it [00:05, 22.87it/s]


# Save results

In [13]:
save_dir = f"evaluation/ar_style_classifier/results/{MODEL_NAME}"
!mkdir -p {save_dir}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
# ── cell: 10 ──
# 💾 Save Results and Metrics
results_df.to_csv(
    f"evaluation/ar_style_classifier/results/{MODEL_NAME}/ar_bsc_evaluation_results_.csv",
    index=False
)

# 🧾 Aggregate metrics per classifier
metrics = results_df.groupby("classifier_id").agg(
    accuracy=("success", "mean"),
    avg_diff=("diff", "mean"),
    count=("success", "size")
).reset_index()

# 🌐 Compute overall metrics across all classifiers and samples
overall_accuracy = results_df["success"].mean()
overall_avg_diff = results_df["diff"].mean()

# Create an overall-metrics DataFrame row
overall_row = pd.DataFrame([{
    "classifier_id": -1,
    "accuracy": overall_accuracy,
    "avg_diff": overall_avg_diff,
    "count": len(results_df)
}])

# Concatenate to include overall stats
metrics = pd.concat([metrics, overall_row], ignore_index=True)

# 🖨 Print overall metrics
print(f"Overall Style-Transfer Accuracy: {overall_accuracy:.4f}")
print(f"Overall Average Diff (strength): {overall_avg_diff:.4f}")

# 💾 Save both per-author and overall metrics
metrics.to_csv(
    f"evaluation/ar_style_classifier/results/{MODEL_NAME}/ar_bsc_metrics_per_author_and_overall.csv",
    index=False
)

print("✅ Saved evaluation and per-author metrics CSVs (including overall metric).")


Overall Style-Transfer Accuracy: 0.5978
Overall Average Diff (strength): 0.0433
✅ Saved evaluation and per-author metrics CSVs (including overall metric).


In [15]:
# ── cell: 11 ──
# 📈 Per-Sample Summary: Identify strongest style match and overall success rate
# For each sample, find the classifier with highest generated_prob
top_matches = results_df.loc[results_df.groupby('sample_id')['generated_prob'].idxmax()]
# top_matches = top_matches.merge(id2author.to_frame('classifier_id').reset_index(), on='classifier_id')
id2author_df = pd.DataFrame(list(id2author.items()), columns=['classifier_id', 'predicted_author'])
top_matches = top_matches.merge(id2author_df, on='classifier_id')
top_matches = top_matches.rename(columns={
    'classifier_id': 'predicted_id',
    'index': 'predicted_author'
})
# Calculate overall success: did the true author classifier succeed?
true_success = results_df[
    (results_df['classifier_id'] == results_df['true_id'])
].groupby('sample_id')['success'].first().reset_index()

# Merge for overview
overview = top_matches.merge(true_success, on='sample_id')

overview.to_csv(f"evaluation/ar_style_classifier/results/{MODEL_NAME}/ar_bsc_evaluation_per_sample_overview.csv", index=False)
print("✅ Saved per-sample overview CSV: strongest match and true-author success.")

# 💡 Note: Probabilities are averaged across all chunks of a sample, so each sample has a single neutral_prob and generated_prob per classifier.

✅ Saved per-sample overview CSV: strongest match and true-author success.


In [16]:
# ── cell: 12 ──
# 📦 Package Results Archive
import os, subprocess

zip_name = f"ar_style_classifier_results_{MODEL_NAME}.zip"
results_dir = os.path.join("evaluation", "ar_style_classifier", "results", MODEL_NAME)

# Run compression
!cd evaluation/ar_style_classifier && zip -r {zip_name} results/{MODEL_NAME}

print(f"✅ Created archive: evaluation/ar_style_classifier/{zip_name}")

# !cd evaluation/ar_style_classifier && zip -r ar_style_classifier_results_{MODEL_NAME}.zip results/{MODEL_NAME}

updating: results/google_mt5-small/ (stored 0%)
updating: results/google_mt5-small/ar_bsc_evaluation_results_.csv (deflated 59%)
updating: results/google_mt5-small/ar_bsc_evaluation_per_sample_overview.csv (deflated 67%)
updating: results/google_mt5-small/ar_bsc_metrics_per_author_and_overall.csv (deflated 48%)
✅ Created archive: evaluation/ar_style_classifier/ar_style_classifier_results_google_mt5-small.zip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
from IPython.display import FileLink

FileLink(f"evaluation/ar_style_classifier/{zip_name}")

/kaggle/working/evaluation/ar_style_classifier/ar_style_classifier_results_google_mt5-small.zip

In [18]:
# # --- Inference on full validation texts ---
# def predict_full_text(text):
#     enc = tokenizer(
#         text,
#         truncation=True,
#         padding="max_length",
#         max_length=512,
#         stride=256,
#         return_overflowing_tokens=True,
#         return_tensors="pt"
#     ).to(device)
#     logits = model(**enc).logits  # [num_chunks, num_labels]
#     avg_logits = logits.mean(dim=0)
#     probs = torch.softmax(avg_logits, dim=0)
#     return torch.argmax(probs).item(), probs.cpu().tolist()

In [19]:
# # --- Evaluate across validation set ---
# preds, refs = [], []
# for example in val_df.itertuples():
#     pred, _ = predict_full_text(example.text_in_author_style)
#     preds.append(pred)
#     refs.append(author2id[example.author])

In [20]:
# from sklearn.metrics import accuracy_score, f1_score
# print("Validation Accuracy:", accuracy_score(refs, preds))
# print("Validation F1‑macro:", f1_score(refs, preds, average="macro"))